In [11]:
from datasets import load_dataset


# 加载HealthCareMagic-100k数据集
dataset = load_dataset(
    'xxxxx/cMedQA2',
    'deduplicate_all',
    trust_remote_code=True,
)

In [12]:
# 查看数据集的信息
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
})

In [13]:
# 查看数据集的列名
dataset.column_names

{'train': ['question', 'answer'],
 'validation': ['question', 'answer'],
 'test': ['question', 'answer']}

In [14]:
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

In [15]:
# # 训练集选择5000条数据，验证集和测试集选择500条数据
# train_dataset = train_dataset.select(range(50000))
# val_dataset = val_dataset.select(range(2000))
# test_dataset = test_dataset.select(range(2000))

In [16]:
# 将数据集保存为alpaca格式
"""alpaca格式
[
    {
        "instruction": "假设您是一名医生，请回答以下选择题（直接给出答案）。",
        "input": "问题：对医疗废物的收集，运转，贮存，处置中的疾病防治工作进行定期检测监督检査的部门是\n选项：\n\t市容监督机关\n\t城市规划行政部门\n\t卫生行政主管部门\n\t检验检疫行政主管部门\n\t环境保护行政部门",
        "output": "卫生行政主管部门"
    },
    {
        "instruction": "假设您是一名医生，请回答以下选择题（直接给出答案）。",
        "input": "问题：既治湿热泻痢，又治感冒发热的药是\n选项：\n\t射干\n\t白头翁\n\t马齿苋\n\t穿心莲\n\t白花蛇舌草",
        "output": "穿心莲"
    },
    {
        "instruction": "假设您是一名医生，请回答以下选择题（直接给出答案）。",
        "input": "问题：主要根据沸点高低分离物质的方法是\n选项：\n\t膜分离法\n\t聚酰胺色谱法\n\t硅胶柱色谱法\n\t离子交换树脂法\n\t分馏法",
        "output": "分馏法"
    },
    {
        "instruction": "假设您是一名医生，请回答以下选择题（直接给出答案）。",
        "input": "问题：不属于眼球屈光装置的结构是\n选项：\n\t角膜\n\t房水\n\t巩膜\n\t玻璃体\n\t晶状体",
        "output": "巩膜"
    }
]
"""

def convert_to_alpaca_format(dataset):
    alpaca_dataset = []
    for data in dataset:
        alpaca_data = {
            "instruction": "如果您是医生，请根据患者的描述回答医疗问题。",
            "input": data['question'],
            "output": data['answer']
        }
        alpaca_dataset.append(alpaca_data)
    return alpaca_dataset

train_alpaca_dataset = convert_to_alpaca_format(train_dataset)
val_alpaca_dataset = convert_to_alpaca_format(val_dataset)
test_alpaca_dataset = convert_to_alpaca_format(test_dataset)


In [17]:
# 保存数据集
import json

def save_dataset(dataset, save_path):
    # 如果文件夹不存在，则创建文件夹
    import os
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)

save_path = f"./data/llama-factory"

save_dataset(train_alpaca_dataset, f"{save_path}/train.json")
save_dataset(val_alpaca_dataset, f"{save_path}/val.json")
save_dataset(test_alpaca_dataset, f"{save_path}/test.json")